In [ ]:
import random
import numpy as np
from deap import base, creator, tools
import matplotlib.pyplot as plt
import math

# Cantidad de segmentos (varillas) del mecanismo
N_SEGMENTS = 11
MIN_LENGTH = 10.0
MAX_LENGTH = 60.0
# Distancia fija entre pivotes de base
PIVOT_DISTANCE = 38.0

# Parámetros de evaluación
ground_threshold = 2.0
continuity_weight = 1.0
ground_contact_weight = 2.0
crossing_penalty = 100.0  # Penalización por varillas que se cruzan

# --- Función para verificar si dos segmentos de línea se cruzan ---
def segments_intersect(p1, p2, p3, p4):
    """
    Verifica si el segmento p1-p2 se cruza con el segmento p3-p4
    Devuelve True si se cruzan, False en caso contrario
    """
    def ccw(a, b, c):
        # Determina si tres puntos están en sentido contrario a las agujas del reloj
        return (c[1] - a[1]) * (b[0] - a[0]) > (b[1] - a[1]) * (c[0] - a[0])

    # Verificar si los segmentos comparten un punto extremo
    if np.array_equal(p1, p3) or np.array_equal(p1, p4) or np.array_equal(p2, p3) or np.array_equal(p2, p4):
        return False  # Los segmentos se conectan en un extremo, no se consideran cruzados

    # El test de cruce de segmentos
    return ccw(p1, p3, p4) != ccw(p2, p3, p4) and ccw(p1, p2, p3) != ccw(p1, p2, p4)

# --- Función auxiliar: intersección de dos círculos con manejo robusto de errores ---
def circle_intersection(p0, r0, p1, r1):
    """Calcula la intersección de dos círculos con manejo robusto de errores numéricos"""
    p0 = np.asarray(p0, float)
    p1 = np.asarray(p1, float)
    d = np.linalg.norm(p1 - p0)

    # Tolerancia numérica para manejar errores de redondeo
    epsilon = 1e-6

    # Si los círculos están demasiado lejos o demasiado cerca
    if d > r0 + r1 + epsilon:
        # Intentar aproximar estirando un poco los radios
        r0_adjusted = r0 * 1.01
        r1_adjusted = r1 * 1.01
        if d > r0_adjusted + r1_adjusted:
            return []  # Realmente no hay intersección posible

    if d < abs(r0 - r1) - epsilon:
        # Intentar aproximar ajustando los radios
        r0_adjusted = min(r0, r1) * 0.99
        r1_adjusted = max(r0, r1) * 0.99
        if d < abs(r0_adjusted - r1_adjusted):
            return []  # Un círculo está completamente dentro del otro

    # Caso de círculos coincidentes
    if abs(d) < epsilon and abs(r0 - r1) < epsilon:
        # Devolver puntos arbitrarios en el círculo
        return [p0 + np.array([r0, 0]), p0 + np.array([-r0, 0])]

    # Fórmula estándar para intersección de círculos
    a = (r0**2 - r1**2 + d**2) / (2 * d)

    # Calcular h con tolerancia numérica
    h_square = r0**2 - a**2
    if h_square < 0:
        if abs(h_square) < epsilon:  # Error numérico pequeño
            h_square = 0
        else:
            # Intentar ajustar los radios ligeramente
            r0_adjusted = r0 * 1.005
            h_square = r0_adjusted**2 - a**2
            if h_square < 0:
                return []

    h = math.sqrt(h_square)

    # Calcular punto base de intersección
    p2 = p0 + a * (p1 - p0) / d

    # Vector perpendicular para los dos puntos de intersección
    perp = h * np.array([-(p1-p0)[1]/d, (p1-p0)[0]/d])

    # Devolver los dos puntos de intersección
    return [p2 + perp, p2 - perp]

# --- Función para inicializar longitudes de varillas con valores inspirados en Theo Jansen ---
def init_theo_jansen():
    """Inicializa con valores cercanos a las proporciones de Theo Jansen"""
    # Estos valores son proporcionales a los del mecanismo original
    # Ajustados para evitar cruces y proporcionar una buena trayectoria
    base = 15.0
    return [
        base * 1.0,    # L1: Manivela
        base * 2.5,    # L2: Acoplador 1
        base * 2.5,    # L3: Chasis a B
        base * 2.0,    # L4: B a C
        base * 2.0,    # L5: Chasis a C
        base * 2.5,    # L6: C a D
        base * 2.5,    # L7: Chasis a D
        base * 2.0,    # L8: D a E
        base * 2.0,    # L9: B a E
        base * 2.0,    # L10: C a F (pie)
        base * 2.0     # L11: E a F (pie)
    ]

# --- Cinemática completa con mejor manejo de errores ---
def full_kinematics(L, theta):
    """Calcula la posición de todos los puntos del mecanismo para un ángulo dado"""
    try:
        O1 = np.array([0.0, 0.0])
        O2 = np.array([PIVOT_DISTANCE, 0.0])

        # A: cigüeñal en O1
        A = np.array([L[0]*math.cos(theta), L[0]*math.sin(theta)])

        # B: (A, L2) con (O2, L3)
        inters = circle_intersection(A, L[1], O2, L[2])
        if not inters:
            return None
        # Seleccionar el punto más alto como B
        B = max(inters, key=lambda p: p[1]) if inters[0][1] != inters[1][1] else inters[0]

        # C: (B, L4) con (O1, L5)
        inters = circle_intersection(B, L[3], O1, L[4])
        if not inters:
            return None
        C = max(inters, key=lambda p: p[1]) if inters[0][1] != inters[1][1] else inters[0]

        # D: (C, L6) con (O2, L7)
        inters = circle_intersection(C, L[5], O2, L[6])
        if not inters:
            return None
        D = max(inters, key=lambda p: p[1]) if inters[0][1] != inters[1][1] else inters[0]

        # E: (D, L8) con (B, L9)
        inters = circle_intersection(D, L[7], B, L[8])
        if not inters:
            return None
        E = max(inters, key=lambda p: p[1]) if inters[0][1] != inters[1][1] else inters[0]

        # F (pie): (C, L10) con (E, L11)
        inters = circle_intersection(C, L[9], E, L[10])
        if not inters:
            return None
        F = max(inters, key=lambda p: p[1]) if inters[0][1] != inters[1][1] else inters[0]

        return {'O1':O1, 'O2':O2, 'A':A, 'B':B, 'C':C, 'D':D, 'E':E, 'F':F}
    except Exception as e:
        # En caso de cualquier error numérico
        return None

# --- Verificar cruces de varillas en el mecanismo ---
def check_rod_crossings(pts):
    """
    Verifica si hay varillas que se cruzan en la configuración dada
    Devuelve el número de cruces detectados
    """
    if not pts:
        return 10  # Penalización alta si no hay configuración válida

    # Definir todos los segmentos (varillas) del mecanismo
    segments = [
        (pts['O1'], pts['A']),  # Manivela
        (pts['A'], pts['B']),   # Acoplador 1
        (pts['B'], pts['O2']),  # Chasis a B
        (pts['B'], pts['C']),   # B a C
        (pts['C'], pts['O1']),  # Chasis a C
        (pts['C'], pts['D']),   # C a D
        (pts['D'], pts['O2']),  # Chasis a D
        (pts['D'], pts['E']),   # D a E
        (pts['B'], pts['E']),   # B a E
        (pts['C'], pts['F']),   # C a F (pie)
        (pts['E'], pts['F'])    # E a F (pie)
    ]

    # Verificar todos los pares de segmentos
    crossings = 0
    for i in range(len(segments)):
        for j in range(i+1, len(segments)):
            # Ignorar segmentos que comparten un extremo (están conectados)
            if (np.array_equal(segments[i][0], segments[j][0]) or
                np.array_equal(segments[i][0], segments[j][1]) or
                np.array_equal(segments[i][1], segments[j][0]) or
                np.array_equal(segments[i][1], segments[j][1])):
                continue

            if segments_intersect(segments[i][0], segments[i][1], segments[j][0], segments[j][1]):
                crossings += 1

    return crossings

# --- Simulación de la trayectoria con detección de cruces ---
def simulate_foot_trajectory(individual, n_steps=72):
    """
    Simula la trayectoria del pie y detecta cruces de varillas
    Devuelve (trayectoria, total_cruces)
    """
    traj = []
    valid_steps = 0
    total_crossings = 0

    thetas = np.linspace(0, 2*math.pi, n_steps)
    for theta in thetas:
        pts = full_kinematics(individual, theta)
        if pts:
            traj.append((pts['F'][0], pts['F'][1]))
            valid_steps += 1

            # Verificar cruces de varillas
            crossings = check_rod_crossings(pts)
            total_crossings += crossings

    # Si menos del 80% de pasos son válidos, la trayectoria no es completa
    if valid_steps < 0.8 * n_steps:
        return [], 100  # Penalización alta por trayectoria incompleta

    return traj, total_crossings

# --- Función de evaluación mejorada con penalización por cruces ---
def evaluate(individual):
    """Evaluación del mecanismo con penalización por cruces de varillas"""
    # Verificar longitudes válidas
    if any(l < MIN_LENGTH or l > MAX_LENGTH for l in individual):
        return -1000.0,

    # Obtener trayectoria y cruces
    traj, total_crossings = simulate_foot_trajectory(individual)
    if len(traj) < 10:  # No suficientes puntos válidos
        return -500.0,

    # Extraer coordenadas
    xs = np.array([p[0] for p in traj])
    ys = np.array([p[1] for p in traj])

    # 1. Longitud del paso
    step_length = xs.max() - xs.min()

    # 2. Altura del paso (queremos que levante el pie)
    step_height = ys.max() - ys.min()

    # 3. Suavidad (penalizar grandes discontinuidades)
    diffs = np.sqrt(np.diff(xs)**2 + np.diff(ys)**2)
    if np.max(diffs) > 10.0:  # Saltos muy grandes
        smoothness = -100
    else:
        smoothness = -np.std(diffs)

    # 4. Contacto con el suelo
    min_y = np.min(ys)
    ground_points = np.sum(ys <= min_y + ground_threshold)
    ground_fraction = ground_points / len(ys)

    # 5. Evaluar cierre de la trayectoria (trayectoria cíclica)
    # Distancia entre el primer y último punto
    closure_distance = np.sqrt((xs[0] - xs[-1])**2 + (ys[0] - ys[-1])**2)
    cycle_quality = -closure_distance  # Negativo porque queremos minimizarlo

    # Calcular fitness combinado
    fitness = (
        step_length * 1.0 +                     # Longitud del paso
        step_height * 0.5 +                     # Altura vertical
        continuity_weight * smoothness +        # Suavidad
        ground_contact_weight * ground_fraction + # Contacto con suelo
        cycle_quality * 2.0 +                   # Trayectoria cerrada
        -crossing_penalty * total_crossings     # Penalización por cruces
    )

    return fitness,

# --- Limpiar definiciones previas de DEAP en caso de reejecutar ---
if 'FitnessMax' in dir(creator):
    del creator.FitnessMax
    del creator.Individual

# --- Setup DEAP ---
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()

# Registrar funciones de inicialización
toolbox.register("attr_float", random.uniform, MIN_LENGTH, MAX_LENGTH)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, N_SEGMENTS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registrar operadores genéticos
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.3)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2.0, indpb=0.25)
toolbox.register("select", tools.selTournament, tournsize=3)

# --- Crear población inicial con variaciones del modelo de Jansen ---
def create_initial_population(pop_size=50):
    """Crea población inicial con varios individuos inspirados en Theo Jansen"""
    population = []

    # Añadir varios individuos basados en el diseño de Theo Jansen
    theo_design = creator.Individual(init_theo_jansen())
    population.append(theo_design)

    # Añadir variaciones del diseño de Jansen
    for _ in range(19):
        ind = creator.Individual(init_theo_jansen())
        # Pequeña variación aleatoria
        for i in range(len(ind)):
            ind[i] *= random.uniform(0.85, 1.15)
        population.append(ind)

    # Completar con individuos aleatorios
    for _ in range(pop_size - len(population)):
        ind = toolbox.individual()
        population.append(ind)

    return population

# --- Parámetros evolutivos ---
POP_SIZE = 80
N_GENERATIONS = 100  # Aumentado para tener más evolución
CXPB, MUTPB = 0.7, 0.3

# --- Inicialización y evolución ---
def run_evolution():
    """Ejecuta el algoritmo evolutivo con una estrategia elitista"""
    # Inicializar población
    pop = create_initial_population(POP_SIZE)

    # Evaluar población inicial
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Registrar estadísticas
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Almacenar los mejores individuos
    hall_of_fame = tools.HallOfFame(5)
    hall_of_fame.update(pop)

    # Registrar progreso
    max_fitness_values = []
    avg_fitness_values = []

    # Mejor individuo sin cruces
    best_no_crossing = None
    best_no_crossing_fitness = float('-inf')

    # Evolución
    for g in range(N_GENERATIONS):
        # Selección con elitismo: conservar los mejores individuos
        elites = list(map(toolbox.clone, hall_of_fame))

        # Selección y clonación para el resto
        offspring = list(map(toolbox.clone, toolbox.select(pop, len(pop) - len(elites))))

        # Cruza
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(c1, c2)
                del c1.fitness.values
                del c2.fitness.values

        # Mutación
        for m in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(m)
                del m.fitness.values

        # Evaluar individuos con fitness invalidado
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Reemplazar población con elitismo
        pop[:] = elites + offspring[:len(pop) - len(elites)]

        # Actualizar hall of fame
        hall_of_fame.update(pop)

        # Obtener estadísticas
        record = stats.compile(pop)
        max_fitness_values.append(record["max"])
        avg_fitness_values.append(record["avg"])

        # Buscar el mejor individuo sin cruces
        for ind in pop:
            _, crossings = simulate_foot_trajectory(ind)
            if crossings == 0 and ind.fitness.values[0] > best_no_crossing_fitness:
                best_no_crossing = toolbox.clone(ind)
                best_no_crossing_fitness = ind.fitness.values[0]
                print(f"¡Nuevo mejor individuo sin cruces encontrado! Fitness: {best_no_crossing_fitness:.2f}")

        # Imprimir progreso
        print(f"Gen {g}: Max={record['max']:.2f}, Avg={record['avg']:.2f}, Válidos={len([i for i in pop if i.fitness.values[0] > 0])}")

        # Visualizar progreso cada 20 generaciones
        if g % 20 == 0 or g == N_GENERATIONS - 1:
            best = hall_of_fame[0]
            traj, crossings = simulate_foot_trajectory(best)
            if traj:
                print(f"  Mejor individuo: Fitness={best.fitness.values[0]:.2f}, Cruces={crossings}")
                plt.figure(figsize=(10, 8))

                # Graficar trayectoria
                xs = [p[0] for p in traj]
                ys = [p[1] for p in traj]
                plt.plot(xs, ys, 'b-', linewidth=2)
                plt.scatter(xs, ys, c=range(len(xs)), cmap='viridis', s=30)

                # Visualizar mecanismo en una posición
                pts = full_kinematics(best, 0)
                if pts:
                    # Dibujar conexiones
                    connections = [
                        ("O1","A"), ("A","B"), ("B","O2"),
                        ("B","C"), ("C","O1"), ("C","D"),
                        ("D","O2"), ("D","E"), ("B","E"),
                        ("C","F"), ("E","F")
                    ]

                    for p, q in connections:
                        x_vals = [pts[p][0], pts[q][0]]
                        y_vals = [pts[p][1], pts[q][1]]
                        plt.plot(x_vals, y_vals, 'r-', linewidth=1.5)

                plt.title(f"Generación {g} - Fitness={best.fitness.values[0]:.2f}, Cruces={crossings}")
                plt.axis('equal')
                plt.grid(True)
                plt.savefig(f"progreso_gen_{g}.png")
                plt.close()

    # Devolver el mejor individuo sin cruces si existe, de lo contrario el mejor general
    if best_no_crossing is not None:
        print("\nSe encontró un mecanismo sin cruces de varillas!")
        return pop, best_no_crossing, max_fitness_values, avg_fitness_values
    else:
        print("\nAdvertencia: No se encontró ningún mecanismo sin cruces de varillas")
        return pop, hall_of_fame[0], max_fitness_values, avg_fitness_values

# --- Visualización del mecanismo en movimiento ---
def visualize_mechanism(individual, title="Mecanismo de Theo Jansen"):
    """Visualiza el mecanismo en varias posiciones y su trayectoria"""
    traj, crossings = simulate_foot_trajectory(individual, n_steps=100)
    if not traj:
        print("No se pudo generar trayectoria válida")
        return False

    # 1. Visualizar trayectoria completa
    xs = [p[0] for p in traj]
    ys = [p[1] for p in traj]

    plt.figure(figsize=(12, 10))

    # Trayectoria
    plt.plot(xs, ys, 'b-', linewidth=2, label="Trayectoria del pie")
    plt.scatter(xs, ys, c=range(len(xs)), cmap='viridis', s=30)
    plt.colorbar(label='Posición en el ciclo')

    # Destacar la posición inicial y final para ver el cierre del ciclo
    plt.plot(xs[0], ys[0], 'go', markersize=8, label="Inicio")
    plt.plot(xs[-1], ys[-1], 'ro', markersize=8, label="Fin")

    plt.title(f"{title} - Trayectoria del pie (Cruces: {crossings})")
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.savefig("trayectoria_completa.png")
    plt.show()

    # 2. Visualizar mecanismo en 6 posiciones clave
    thetas = np.linspace(0, 2*math.pi, 6, endpoint=False)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    axs = axs.flatten()

    # Colores para las varillas para facilitar identificación
    colors = {
        ("O1","A"): 'red',       # Manivela
        ("A","B"): 'orange',     # Acoplador 1
        ("B","O2"): 'yellow',    # Fijo a B
        ("B","C"): 'green',      # B a C
        ("C","O1"): 'cyan',      # Fijo a C
        ("C","D"): 'blue',       # C a D
        ("D","O2"): 'purple',    # Fijo a D
        ("D","E"): 'magenta',    # D a E
        ("B","E"): 'brown',      # B a E
        ("C","F"): 'lime',       # C a F (pie)
        ("E","F"): 'pink'        # E a F (pie)
    }

    for i, theta in enumerate(thetas):
        pts = full_kinematics(individual, theta)
        if not pts:
            print(f"Configuración inválida en theta={theta}")
            axs[i].text(0.5, 0.5, f"Configuración inválida en θ={theta:.2f}",
                       ha='center', va='center', transform=axs[i].transAxes)
            continue

        # Dibujar trayectoria completa en cada subgráfico
        axs[i].plot(xs, ys, 'b--', alpha=0.3)

        # Dibujar posición actual del pie
        axs[i].plot(pts['F'][0], pts['F'][1], 'ro', markersize=8)

        # Dibujar conexiones con colores específicos
        connections = [
            ("O1","A"), ("A","B"), ("B","O2"),
            ("B","C"), ("C","O1"), ("C","D"),
            ("D","O2"), ("D","E"), ("B","E"),
            ("C","F"), ("E","F")
        ]

        for p, q in connections:
            x_vals = [pts[p][0], pts[q][0]]
            y_vals = [pts[p][1], pts[q][1]]
            axs[i].plot(x_vals, y_vals, '-', color=colors[(p,q)], linewidth=2)

        # Dibujar puntos
        for key, point in pts.items():
            axs[i].plot(point[0], point[1], 'o', markersize=6,
                       markerfacecolor='white', markeredgecolor='black')
            axs[i].text(point[0], point[1], key, fontsize=9)

        # Verificar cruces para esta posición
        current_crossings = check_rod_crossings(pts)

        axs[i].set_title(f"θ = {theta*180/math.pi:.1f}° (Cruces: {current_crossings})")
        axs[i].grid(True)
        axs[i].axis('equal')

    plt.tight_layout()
    plt.savefig("mecanismo_posiciones.png")
    plt.show()

    # 3. Animación del mecanismo (versión estática para todos los puntos)
    plt.figure(figsize=(15, 12))
    plt.axis('equal')
    plt.grid(True)

    # Graficar la trayectoria completa
    plt.plot(xs, ys, 'b--', alpha=0.5, label="Trayectoria del pie")

    # Colores para las posiciones del mecanismo
    cmap = plt.cm.viridis

    # Dibujar el mecanismo en múltiples posiciones
    n_frames = 10  # Número de posiciones a mostrar
    thetas = np.linspace(0, 2*math.pi, n_frames, endpoint=False)

    # Conexiones a dibujar
    connections = [
        ("O1","A"), ("A","B"), ("B","O2"),
        ("B","C"), ("C","O1"), ("C","D"),
        ("D","O2"), ("D","E"), ("B","E"),
        ("C","F"), ("E","F")
    ]

    for i, theta in enumerate(thetas):
        pts = full_kinematics(individual, theta)
        if not pts:
            continue

        # Color según la posición en el ciclo
        color = cmap(i / n_frames)
        alpha = 0.5 if i > 0 else 0.8  # Primera posición más visible

        # Dibujar varillas
        for p, q in connections:
            x_vals = [pts[p][0], pts[q][0]]
            y_vals = [pts[p][1], pts[q][1]]
            plt.plot(x_vals, y_vals, '-', color=color, alpha=alpha, linewidth=1)

        # Destacar el pie
        plt.plot(pts['F'][0], pts['F'][1], 'o', color=color, markersize=8)

    # Añadir puntos fijos
    plt.plot(0, 0, 'ks', markersize=10, label="O1 (Pivote fijo)")
    plt.plot(PIVOT_DISTANCE, 0, 'ks', markersize=10, label="O2 (Pivote fijo)")

    plt.title(f"Movimiento del {title} en un ciclo completo")
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.savefig("mecanismo_ciclo_completo.png")
    plt.show()

    return True

def verificar_mecanismo(individual):
    """Verifica si un mecanismo individual funciona correctamente y muestra sus métricas"""
    print("\nVerificando mecanismo...")
    print("Longitudes:", [f"{x:.2f}" for x in individual])

    # Probar cinemática en varias posiciones
    valid_count = 0
    crossings_total = 0

    for theta in np.linspace(0, 2*math.pi, 36):
        result = full_kinematics(individual, theta)
        if result:
            valid_count += 1
            crossings = check_rod_crossings(result)
            crossings_total += crossings

    print(f"Posiciones válidas: {valid_count}/36")
    print(f"Cruces de varillas promedio: {crossings_total/valid_count if valid_count else 'N/A'}")

    # Simular trayectoria
    traj, total_crossings = simulate_foot_trajectory(individual, n_steps=72)
    if not traj:
        print("No se pudo generar una trayectoria válida")
        return False

    print(f"Puntos de trayectoria: {len(traj)}")
    print(f"Total de cruces en la trayectoria: {total_crossings}")

    # Calcular métricas
    xs = [p[0] for p in traj]
    ys = [p[1] for p in traj]

    step_length = max(xs) - min(xs)
    step_height = max(ys) - min(ys)

    print(f"Longitud del paso: {step_length:.2f}")
    print(f"Altura del paso: {step_height:.2f}")

    # Evaluar la suavidad
    diffs = np.sqrt(np.diff(xs)**2 + np.diff(ys)**2)
    print(f"Suavidad (desv. estándar): {np.std(diffs):.4f}")

    # Evaluar cierre de la trayectoria
    closure_distance = np.sqrt((xs[0] - xs[-1])**2 + (ys[0] - ys[-1])**2)
    print(f"Cierre de trayectoria (distancia inicio-fin): {closure_distance:.4f}")

    # Mostrar fitness
    fitness = evaluate(individual)[0]
    print(f"Fitness total: {fitness:.2f}")

    return len(traj) > 0

# --- Función para crear una visualización en HTML/SVG interactiva ---
def create_interactive_visualization(individual, filename="InteractiveOutput.html"):
    """Crea una visualización interactiva del mecanismo en formato HTML/SVG"""
    # Generar trayectoria
    traj, crossings = simulate_foot_trajectory(individual, n_steps=100)
    if not traj:
        print("No se pudo generar trayectoria válida para visualización interactiva")
        return False

    # Extraer coordenadas
    xs = [p[0] for p in traj]
    ys = [p[1] for p in traj]

    # Preparar datos para la visualización
    # Crear HTML con SVG interactivo
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Mecanismo de Theo Jansen Interactivo</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            #container { width: 900px; margin: 0 auto; }
            #mechanism { width: 100%; height: 500px; border: 1px solid #ccc; }
            .controls { margin: 20px 0; }
            .slider { width: 70%; }
            svg { background-color: #f9f9f9; }
            .rod { stroke-width: 3; stroke-linecap: round; }
            .joint { fill: white; stroke: black; stroke-width: 2; }
            .fixed-joint { fill: black; stroke: black; stroke-width: 2; }
            .foot { fill: red; stroke: black; stroke-width: 2; }
            .trajectory { fill: none; stroke: blue; stroke-width: 1.5; stroke-dasharray: 3,3; }
            .infotext { font-size: 14px; font-family: monospace; }
        </style>
    </head>
    <body>
        <div id="container">
            <h1>Simulación de Mecanismo de Theo Jansen</h1>
            <div id="mechanism"></div>
            <div class="controls">
                <button id="playPauseBtn">▶ Iniciar</button>
                <button id="resetBtn">⟲ Reiniciar</button>
                <input type="range" min="0" max="99" value="0" class="slider" id="angleSlider">
                <span id="angleValue">0°</span>
            </div>
            <div id="info">
                <p>Longitudes:</p>
                <pre id="lengths"></pre>
                <p>Cruces detectados: <span id="crossings"></span></p>
            </div>
        </div>

        <script>
            // Datos del mecanismo
            const pivot_distance = """ + str(PIVOT_DISTANCE) + """;
            const segment_lengths = """ + str(individual) + """;

            // Crear el SVG
            const svgNS = "http://www.w3.org/2000/svg";
            const svg = document.createElementNS(svgNS, "svg");
            svg.setAttribute("viewBox", "-20 -100 200 150");
            document.getElementById("mechanism").appendChild(svg);

            // Añadir la trayectoria
            const trajectory = document.createElementNS(svgNS, "path");
            trajectory.setAttribute("class", "trajectory");

            // Puntos de trayectoria
            const trajectoryPoints = """ + str(traj) + """;
            let pathData = "M " + trajectoryPoints[0][0] + " " + -trajectoryPoints[0][1];
            for (let i = 1; i < trajectoryPoints.length; i++) {
                pathData += " L " + trajectoryPoints[i][0] + " " + -trajectoryPoints[i][1];
            }
            trajectory.setAttribute("d", pathData);
            svg.appendChild(trajectory);

            // Crear elementos para el mecanismo
            const rods = [];
            const rodConnections = [
                ["O1", "A"], ["A", "B"], ["B", "O2"],
                ["B", "C"], ["C", "O1"], ["C", "D"],
                ["D", "O2"], ["D", "E"], ["B", "E"],
                ["C", "F"], ["E", "F"]
            ];

            // Colores para las varillas
            const rodColors = [
                "red", "orange", "yellow",
                "green", "cyan", "blue",
                "purple", "magenta", "brown",
                "lime", "pink"
            ];

            // Crear las varillas
            for (let i = 0; i < rodConnections.length; i++) {
                const rod = document.createElementNS(svgNS, "line");
                rod.setAttribute("class", "rod");
                rod.setAttribute("stroke", rodColors[i]);
                svg.appendChild(rod);
                rods.push(rod);
            }

            // Crear las articulaciones
            const joints = {
                "O1": createJoint(0, 0, "fixed-joint", "O1"),
                "O2": createJoint(pivot_distance, 0, "fixed-joint", "O2"),
                "A": createJoint(0, 0, "joint", "A"),
                "B": createJoint(0, 0, "joint", "B"),
                "C": createJoint(0, 0, "joint", "C"),
                "D": createJoint(0, 0, "joint", "D"),
                "E": createJoint(0, 0, "joint", "E"),
                "F": createJoint(0, 0, "foot", "F")
            };

            // Mostrar longitudes
            document.getElementById("lengths").textContent = segment_lengths.map((l, i) => `L${i+1}: ${l.toFixed(2)}`).join("\\n");

            // Variables para animación
            let angle = 0;
            let animationId = null;
            let isPlaying = false;

            // Funciones para cinemática
            function updateMechanism(angle) {
                const points = calculatePoints(angle);
                if (!points) {
                    console.error("Configuración inválida para ángulo: " + angle);
                    return false;
                }

                // Actualizar posiciones de las articulaciones
                for (const [key, point] of Object.entries(points)) {
                    if (key === "O1" || key === "O2") continue; // Puntos fijos
                    joints[key].setAttribute("cx", point[0]);
                    joints[key].setAttribute("cy", -point[1]); // Invertir Y para SVG
                }

                // Actualizar posiciones de las varillas
                for (let i = 0; i < rodConnections.length; i++) {
                    const [p1, p2] = rodConnections[i];
                    rods[i].setAttribute("x1", points[p1][0]);
                    rods[i].setAttribute("y1", -points[p1][1]);
                    rods[i].setAttribute("x2", points[p2][0]);
                    rods[i].setAttribute("y2", -points[p2][1]);
                }

                // Detectar cruces
                const crossings = countRodCrossings(points);
                document.getElementById("crossings").textContent = crossings;

                return true;
            }

            function createJoint(x, y, className, label) {
                const joint = document.createElementNS(svgNS, "circle");
                joint.setAttribute("class", className);
                joint.setAttribute("cx", x);
                joint.setAttribute("cy", -y);
                joint.setAttribute("r", 3);
                svg.appendChild(joint);

                const text = document.createElementNS(svgNS, "text");
                text.setAttribute("class", "infotext");
                text.setAttribute("x", x + 5);
                text.setAttribute("y", -y - 5);
                text.textContent = label;
                svg.appendChild(text);

                return joint;
            }

            function circleIntersection(p0, r0, p1, r1) {
                const epsilon = 1e-6;
                let dx = p1[0] - p0[0];
                let dy = p1[1] - p0[1];
                const d = Math.sqrt(dx*dx + dy*dy);

                // Verificar si los círculos están demasiado lejos o demasiado cerca
                if (d > r0 + r1 + epsilon || d < Math.abs(r0 - r1) - epsilon) {
                    return [];
                }

                // Caso de círculos coincidentes
                if (Math.abs(d) < epsilon && Math.abs(r0 - r1) < epsilon) {
                    return [[p0[0] + r0, p0[1]], [p0[0] - r0, p0[1]]];
                }

                // Cálculo de los puntos de intersección
                const a = (r0*r0 - r1*r1 + d*d) / (2*d);
                const h = Math.sqrt(r0*r0 - a*a);

                const p2x = p0[0] + a * (p1[0] - p0[0]) / d;
                const p2y = p0[1] + a * (p1[1] - p0[1]) / d;

                const p3x1 = p2x + h * (p1[1] - p0[1]) / d;
                const p3y1 = p2y - h * (p1[0] - p0[0]) / d;

                const p3x2 = p2x - h * (p1[1] - p0[1]) / d;
                const p3y2 = p2y + h * (p1[0] - p0[0]) / d;

                return [[p3x1, p3y1], [p3x2, p3y2]];
            }

            function calculatePoints(theta) {
                try {
                    const O1 = [0, 0];
                    const O2 = [pivot_distance, 0];

                    // A: manivela en O1
                    const A = [
                        segment_lengths[0] * Math.cos(theta),
                        segment_lengths[0] * Math.sin(theta)
                    ];

                    // B: intersección de círculos (A, L2) y (O2, L3)
                    const intersB = circleIntersection(A, segment_lengths[1], O2, segment_lengths[2]);
                    if (intersB.length === 0) return null;
                    // Seleccionar el punto más alto
                    const B = intersB[0][1] > intersB[1][1] ? intersB[0] : intersB[1];

                    // C: intersección de círculos (B, L4) y (O1, L5)
                    const intersC = circleIntersection(B, segment_lengths[3], O1, segment_lengths[4]);
                    if (intersC.length === 0) return null;
                    const C = intersC[0][1] > intersC[1][1] ? intersC[0] : intersC[1];

                    // D: intersección de círculos (C, L6) y (O2, L7)
                    const intersD = circleIntersection(C, segment_lengths[5], O2, segment_lengths[6]);
                    if (intersD.length === 0) return null;
                    const D = intersD[0][1] > intersD[1][1] ? intersD[0] : intersD[1];

                    // E: intersección de círculos (D, L8) y (B, L9)
                    const intersE = circleIntersection(D, segment_lengths[7], B, segment_lengths[8]);
                    if (intersE.length === 0) return null;
                    const E = intersE[0][1] > intersE[1][1] ? intersE[0] : intersE[1];

                    // F: intersección de círculos (C, L10) y (E, L11)
                    const intersF = circleIntersection(C, segment_lengths[9], E, segment_lengths[10]);
                    if (intersF.length === 0) return null;
                    const F = intersF[0][1] > intersF[1][1] ? intersF[0] : intersF[1];

                    return {
                        "O1": O1, "O2": O2, "A": A, "B": B,
                        "C": C, "D": D, "E": E, "F": F
                    };

                } catch (e) {
                    console.error("Error en cálculo de cinemática:", e);
                    return null;
                }
            }

            function ccw(a, b, c) {
                return (c[1] - a[1]) * (b[0] - a[0]) > (b[1] - a[1]) * (c[0] - a[0]);
            }

            function segmentsIntersect(p1, p2, p3, p4) {
                // Ignorar segmentos conectados
                if ((p1[0] === p3[0] && p1[1] === p3[1]) ||
                    (p1[0] === p4[0] && p1[1] === p4[1]) ||
                    (p2[0] === p3[0] && p2[1] === p3[1]) ||
                    (p2[0] === p4[0] && p2[1] === p4[1])) {
                    return false;
                }
                return ccw(p1, p3, p4) !== ccw(p2, p3, p4) && ccw(p1, p2, p3) !== ccw(p1, p2, p4);
            }

            function countRodCrossings(pts) {
                let crossings = 0;
                const segments = rodConnections.map(([p1, p2]) => [pts[p1], pts[p2]]);

                for (let i = 0; i < segments.length; i++) {
                    for (let j = i+1; j < segments.length; j++) {
                        if (segmentsIntersect(segments[i][0], segments[i][1], segments[j][0], segments[j][1])) {
                            crossings++;
                        }
                    }
                }

                return crossings;
            }

            // Controles de interfaz
            const slider = document.getElementById("angleSlider");
            const angleValueDisplay = document.getElementById("angleValue");
            const playPauseBtn = document.getElementById("playPauseBtn");
            const resetBtn = document.getElementById("resetBtn");

            // Inicializar mecanismo
            updateMechanism(0);

            // Eventos
            slider.oninput = function() {
                angle = (this.value / 50) * Math.PI;
                angleValueDisplay.textContent = Math.round((angle * 180 / Math.PI)) + "°";
                updateMechanism(angle);
            };

            playPauseBtn.onclick = function() {
                if (isPlaying) {
                    cancelAnimationFrame(animationId);
                    playPauseBtn.textContent = "▶ Iniciar";
                } else {
                    animate();
                    playPauseBtn.textContent = "⏸ Pausar";
                }
                isPlaying = !isPlaying;
            };

            resetBtn.onclick = function() {
                if (isPlaying) {
                    cancelAnimationFrame(animationId);
                    playPauseBtn.textContent = "▶ Iniciar";
                    isPlaying = false;
                }
                angle = 0;
                slider.value = 0;
                angleValueDisplay.textContent = "0°";
                updateMechanism(angle);
            };

            function animate() {
                angle += 0.02;
                if (angle > 2 * Math.PI) {
                    angle = 0;
                }
                slider.value = (angle / (2 * Math.PI)) * 100;
                angleValueDisplay.textContent = Math.round((angle * 180 / Math.PI)) + "°";
                updateMechanism(angle);
                animationId = requestAnimationFrame(animate);
            }
        </script>
    </body>
    </html>
    """

    # Escribir a archivo
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Visualización interactiva guardada en '{filename}'")
    return True

# --- Utilizar proporciones conocidas de Theo Jansen ---
theo_jansen_orig = init_theo_jansen()
print("\nVerificando diseño original de Theo Jansen...")
verificar_mecanismo(theo_jansen_orig)

# Visualizar el mecanismo original de Theo Jansen
print("\nVisualizando mecanismo original de Theo Jansen...")
visualize_mechanism(theo_jansen_orig, "Mecanismo Original de Theo Jansen")

# Crear visualización interactiva del mecanismo original
print("\nCreando visualización interactiva del mecanismo original...")
create_interactive_visualization(theo_jansen_orig, "mecanismo_original_interactivo.html")

# --- Ejecutar evolución si se desea encontrar mejores versiones ---
print("\n¿Desea ejecutar el algoritmo evolutivo para optimizar el mecanismo? (s/n)")
run_evo = input() if 'get_ipython' not in globals() else 's'

if run_evo.lower() == 's':
    print("\nIniciando evolución...")
    pop, best, max_fitness, avg_fitness = run_evolution()

    # Mostrar resultados
    print("\nMejor individuo encontrado:")
    print("Fitness:", best.fitness.values[0])
    print("Longitudes:", [f"{x:.2f}" for x in best])

    # Visualizar evolución del fitness
    plt.figure(figsize=(10, 6))
    plt.plot(max_fitness, label='Máximo')
    plt.plot(avg_fitness, label='Promedio')
    plt.title("Evolución del Fitness")
    plt.xlabel("Generación")
    plt.ylabel("Fitness")
    plt.legend()
    plt.grid(True)
    plt.savefig("evolucion_fitness.png")
    plt.show()

    # Verificar y visualizar el mejor mecanismo encontrado
    print("\nVerificando el mejor mecanismo encontrado...")
    verificar_mecanismo(best)

    print("\nVisualizando mejor mecanismo encontrado...")
    visualize_mechanism(best, "Mecanismo Optimizado")

    # Crear visualización interactiva del mecanismo optimizado
    print("\nCreando visualización interactiva del mecanismo optimizado...")
    create_interactive_visualization(best, "mecanismo_optimizado_interactivo.html")

    # Comparar con el mecanismo original
    print("\nComparando trayectorias:")
    # Obtener trayectorias
    traj_orig, cross_orig = simulate_foot_trajectory(theo_jansen_orig)
    traj_best, cross_best = simulate_foot_trajectory(best)

    if traj_orig and traj_best:
        plt.figure(figsize=(12, 8))

        # Graficar trayectoria original
        xs_orig = [p[0] for p in traj_orig]
        ys_orig = [p[1] for p in traj_orig]
        plt.plot(xs_orig, ys_orig, 'b-', linewidth=2, label=f"Original (Cruces: {cross_orig})")

        # Graficar trayectoria optimizada
        xs_best = [p[0] for p in traj_best]
        ys_best = [p[1] for p in traj_best]
        plt.plot(xs_best, ys_best, 'r-', linewidth=2, label=f"Optimizado (Cruces: {cross_best})")

        plt.title("Comparación de Trayectorias")
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.legend()
        plt.grid(True)
        plt.axis('equal')
        plt.savefig("comparacion_trayectorias.png")
        plt.show()

    # Guardar el mejor individuo
    import pickle
    with open('best_theo_jansen_mechanism_no_crossings.pkl', 'wb') as f:
        pickle.dump(best, f)
    print("\nMecanismo optimizado guardado en 'best_theo_jansen_mechanism_no_crossings.pkl'")
else:
    print("\nUtilizando solo el mecanismo original de Theo Jansen.")

# Para cargar un mecanismo guardado previamente:
# import pickle
# with open('best_theo_jansen_mechanism_no_crossings.pkl', 'rb') as f:
#     loaded_mechanism = pickle.load(f)
# visualize_mechanism(loaded_mechanism, "Mejor Mecanismo Cargado")